<a href="https://colab.research.google.com/github/Rohawaseem/Task-2/blob/main/week_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import files
import pandas as pd


uploaded = files.upload()

# 2️⃣ Get the filename
filename = list(uploaded.keys())[0]

# 3️⃣ Load the CSV
df = pd.read_csv(filename)

# 4️⃣ Inspect data
print(df.head())
print(df.columns)
print(df.info())


Saving twitter data.csv to twitter data.csv
    id  label                                              tweet
0  1.0    0.0   @user when a father is dysfunctional and is s...
1  2.0    0.0  @user @user thanks for #lyft credit i can't us...
2  3.0    0.0                                bihday your majesty
3  4.0    0.0  #model   i love u take with u all the time in ...
4  5.0    0.0             factsguide: society now    #motivation
Index(['id', 'label', 'tweet'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      31378 non-null  float64
 1   label   31378 non-null  float64
 2   tweet   31378 non-null  object 
dtypes: float64(2), object(1)
memory usage: 749.2+ KB
None


In [4]:
def clean_text(text):
    text = str(text)
    text = re.sub(r'http\S+', '', text)            # remove URLs
    text = re.sub(r'@\w+', '', text)               # remove mentions
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)     # remove hashtags
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)     # remove punctuation
    text = text.lower().strip()
    return text

df['cleaned'] = df['tweet'].apply(clean_text)


In [5]:
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['cleaned_nostop'] = df['cleaned'].apply(remove_stopwords)


In [6]:
# Simple whitespace tokenization
df['tokens'] = df['cleaned_nostop'].apply(lambda x: x.split())

print(df[['cleaned_nostop','tokens']].head())


                                      cleaned_nostop  \
0  father dysfunctional selfish drags kids dysfun...   
1  thanks credit cant use cause dont offer wheelc...   
2                                     bihday majesty   
3                              love u take u time ur   
4                                 factsguide society   

                                              tokens  
0  [father, dysfunctional, selfish, drags, kids, ...  
1  [thanks, credit, cant, use, cause, dont, offer...  
2                                  [bihday, majesty]  
3                       [love, u, take, u, time, ur]  
4                              [factsguide, society]  


In [7]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=1000)  # limit features for performance

# Fit & transform text
X_tfidf = tfidf.fit_transform(df['cleaned_nostop'])

print("TF-IDF Shape:", X_tfidf.shape)


TF-IDF Shape: (31962, 1000)


In [11]:
# 1️⃣ Inspect label column
print(df['label'].head(10))
print(df['label'].dtype)
print(df['label'].isnull().sum())  # Check missing labels

# 2️⃣ Map only if dtype is object (string)
label_map = {'negative':0, 'neutral':1, 'positive':2}

if df['label'].dtype == object:
    # Convert to lowercase & map
    df['label_code'] = df['label'].str.lower().map(label_map)
else:
    # Already numeric, use directly
    df['label_code'] = df['label']

# 3️⃣ Drop rows where mapping failed
df = df.dropna(subset=['label_code']).reset_index(drop=True)

# 4️⃣ Ensure integer type
df['label_code'] = df['label_code'].astype(int)

# 5️⃣ Assign y for model
y = df['label_code'].values

# Check
print("Labels shape:", y.shape)
print("Unique labels:", df['label_code'].unique())


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: label, dtype: float64
float64
584
Labels shape: (31378,)
Unique labels: [0 1]
